In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [4]:
print(os.getcwd())

/workspaces/Product-inventory-analytics


## Проведем аналогичные преобразования

In [6]:
Oststki = pd.read_excel('Остатки.xlsx', header=10)

# Заменили пустые значения нулями
Oststki = Oststki.fillna(0)

Oststki.drop(columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace = True)

Oststki.rename(columns = {'Номенклатура, № по кат., Производитель':'Name', 
                          'Количество (в ед. хранения)':'Нач_ост_кол',
                          'Сумма (в рег. валюте)':'Нач_ост_сумм',
                          'Количество (в ед. хранения).1':'Приход_Кол',
                          'Сумма (в рег. валюте).1':'Приход_Сумм',
                          'Количество (в ед. хранения).2':'Расход_кол',
                          'Сумма (в рег. валюте).2':'Расход_сумм',
                          'Количество (в ед. хранения).3':'Кон_ост_кол',
                          'Сумма (в рег. валюте).3':'Кон_ост_сумм',
                          }, inplace = True )

In [7]:
# Отберем данные для отдельной таблице по Угданскому Чатц
start = Oststki.index [Oststki['Name']== "Угданский ЧАТЦ  склад " ]. tolist ()[0]
end = Oststki.index [Oststki['Name']== "Основной ЧАТЦ" ]. tolist ()[0]

In [8]:
# Функция для отбора только данным по Угданский Чатц
list = [i for i in range(start, end+1)]
Ugdan = Oststki.take(list, axis = 0)

## Анализ групп

In [9]:
# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
rule_2 = ["ЧАТЦ"]
Ugdan = Ugdan[~Ugdan.Name.str.contains('|'.join(rule))]
Ugdan = Ugdan[~Ugdan.Name.str.contains('|'.join(rule_2))]

In [10]:
Ugdan.tail()

,Name,Нач_ост_кол,Нач_ост_сумм,Приход_Кол,Приход_Сумм,Расход_кол,Расход_сумм,Кон_ост_кол,Кон_ост_сумм
8509,"Шнур AUX (цветной) 3 м 16807, 16807,",1.0,82.00,0.0,0.00,1.0,82.00,0.0,0.0
8510,Шприц рычажно-плунжерный 500 мл БелАК БАК00019...,6.0,4070.88,7.0,5009.93,9.0,6217.41,4.0,2863.4
8511,"Штeкepный paзъeм, abs Sampa 095159, 095159, Orex",0.0,0.00,2.0,1946.00,0.0,0.00,2.0,1946.0
8512,"Эпоксидный клей усиленный AVS AVK-129 17922, 1...",3.0,369.00,4.0,764.00,7.0,1133.00,0.0,0.0
8513,"Эпоксидный клей,универсальный, 6мл. ЭДП 24595,...",0.0,0.00,10.0,1270.00,2.0,254.00,8.0,1016.0


### Информация по группам

In [11]:
# Группа 1

Count_group_1 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0')['Name'].count()
Sum_group_1 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0')['Кон_ост_сумм'].sum()

Count_group_1_5 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')['Name'].count()
Sum_group_1_5 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')['Кон_ост_сумм'].sum()

Count_group_1_10 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')['Name'].count()
Sum_group_1_10 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')['Кон_ост_сумм'].sum()

In [12]:
# Группа 2
Count_group_2 = Ugdan.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')['Name'].count()
Sum_group_2 = Ugdan.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')['Кон_ост_сумм'].sum()

# Группа 3
Count_group_3 = Ugdan.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_кол > Расход_кол')['Name'].count()
Sum_group_3 = Ugdan.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_кол > Расход_кол')['Кон_ост_сумм'].sum()

# Группа 4
Count_group_4 = Ugdan.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')['Name'].count()
Sum_group_4 = Ugdan.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')['Кон_ост_сумм'].sum()

# Группа 5
Count_group_5 = Ugdan.query('Расход_кол >=3 and Кон_ост_кол == 0')['Name'].count()
Sum_group_5 = Ugdan.query('Расход_кол >=3 and Кон_ост_кол == 0')['Расход_сумм'].sum()

In [13]:
groups = pd.DataFrame({'Наименование': ['группа 1- "Мертвый" склад', 'группа 1 - свыше 5 тыс', 'группа 1 - свыше 10 тыс', 'группа 2 - Бестолковые закупки', 'группа 3 - Излишние запасы', 'группа 4 - Проверка спроса', 'группа 5 - Нет ходового товара'], 
                    'Сколько позиций.': [Count_group_1, Count_group_1_5, Count_group_1_10, Count_group_2, Count_group_3, Count_group_4, Count_group_5], 
                    'Сумма.': [Sum_group_1, Sum_group_1_5, Sum_group_1_10, Sum_group_2, Sum_group_3, Sum_group_4, Sum_group_5]})
groups = groups.set_index('Наименование').round(2)

groups

,Сколько позиций.,Сумма.
Наименование,,
"группа 1- ""Мертвый"" склад",540,2244497.02
группа 1 - свыше 5 тыс,79,840545.93
группа 1 - свыше 10 тыс,25,370146.40
группа 2 - Бестолковые закупки,3,17927.73
группа 3 - Излишние запасы,138,627833.31
группа 4 - Проверка спроса,142,800914.26
группа 5 - Нет ходового товара,142,891477.19


## Создадим отдельные таблицы для более датального просмотра товарных позиций

In [14]:
Group_1 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0')
Group_1_5 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 5000')
Group_1_10 = Ugdan.query('Приход_Кол == 0 and Расход_кол == 0 and (Кон_ост_сумм/Кон_ост_кол) > 10000')

Group_2 = Ugdan.query('Нач_ост_сумм > 0 and Приход_Сумм > 0 and Расход_сумм == 0')
Group_3 = Ugdan.query('Нач_ост_сумм > 0 and Расход_сумм > 0 and Кон_ост_сумм > Расход_сумм')
Group_4 = Ugdan.query('Нач_ост_сумм == 0 and  Приход_Сумм > 0 and Расход_сумм == 0')
Group_5 = Ugdan.query('Расход_кол >=3 and Кон_ост_кол == 0')

# Выгрузка итоговой таблицы

In [15]:
# Отредактируем файл Metrics для добавления обновленных листов
wb = openpyxl.load_workbook('Metrics.xlsx') 

In [16]:
sheet = wb.sheetnames
print(sheet)

['Sheet1']


In [ ]:
with pd.ExcelWriter('Metrics.xlsx', mode='a', if_sheet_exists= 'replace') as writer:  
    Group_1.to_excel(writer, sheet_name='Группа 1',index = False)
    Group_1_5.to_excel(writer, sheet_name='Группа 1 (>5)',index = False)
    Group_1_10.to_excel(writer, sheet_name='Группа 1 (>10)',index = False)
    Group_2.to_excel(writer, sheet_name='Группа 2',index = False)
    Group_3.to_excel(writer, sheet_name='Группа 3',index = False)
    Group_4.to_excel(writer, sheet_name='Группа 4',index = False)
    Group_5.to_excel(writer, sheet_name='Группа 5',index = False)